In [2]:
#Scripts to maintain database

In [32]:
# Dependencies
# ----------------------------------
from sqlalchemy import create_engine, func # Imports the method used for connecting to DBs
from sqlalchemy.ext.declarative import declarative_base # Imports the methods needed to abstract classes into tables
from sqlalchemy import Column, Integer, String, Float, ForeignKey # Allow us to declare column types
from sqlalchemy.orm import relationship
import csv
import pandas as pd #drastically eases 

compTemplPath = "../resources/templates/componentTemplate.csv"

In [7]:
#CREATE TABLES

# #CREATE TABLE TEMPLATE
# class Item(Base):
#     __tablename__ = 'item'
#     id = Column(Integer, primary_key=True)
#     float = Column(Float(2))
#     string = Column(String(255))
#     int = Column(Integer)

# # Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class Item(Base):
    __tablename__ = 'item'
    id = Column(Integer, primary_key=True) #Change this to itemId
    price = Column(Float(2))
    name = Column(String(255))
    kind = Column(Integer)
    updtDtTm = Column(String(20))
    #baseQtyPerHr = Column(Float(2)) #Need to add production per hour per level
    
class compReltn(Base):
    __tablename__ = 'component_reltn'
    comp_reltn_id = Column(Integer, primary_key=True)
    parent_item_id = Column(Integer, ForeignKey(Item.id))
    comp_item_id = Column(Integer, ForeignKey(Item.id))
    component_qty = Column(Integer)

parentItem = relationship('Item', foreign_keys='compReltn.parent_item_id') #Parent Id Relation
compItem = relationship('Item', foreign_keys='compReltn.comp_item_id') #Component Id Relation
    


In [8]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///../resources/db/db.sqlite")
conn = engine.connect()

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
#Base.metadata.drop_all(engine)

In [25]:
# Create a Session Object to Connect to DB
# ----------------------------------
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [85]:
#Import component template data
# Template contains data of each item's components, and their respective quantities
# NOTE: This could place heavy loads on SQL, as it's making a BUNCH of micro calls. Might give this a bit
#    to run and be mindful of potential slowness elsewhere

#Read in CSV file, append to arroy
newRow = []
templateData = []

with open(compTemplPath, newline='') as csvfile:
    csvfile.seek(0)
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader: #Loop through csv file
        for col in row: #Each column within row
            newRow.append(col) #Append column
        templateData.append(newRow) #When done with row, append to data array
        newRow = [] #Clear out row list

#For each item, we must do the following:
    # DONE - Look up parent item (first item listed) based on name
        # If not found, skip the entire row. We can't go any further with this item. Log this out
    # DONE - Look up component item - similar to above. Need to get this, otherwise skip addition
    # Finally, we need to check if this relationship between parent and component already exists.
        # If no, proceed to add
        # If yes, check if qty matches - if so, skip; if no, update
    
for row in templateData:
    parentName = row[0].upper()
    compName = row[1].upper()
    compQty = float(row[2])
    #print(parentName)
    
    #Search for parent item in table
    exists = session.query(Item).filter(Item.name.ilike(parentName)).scalar() is not None
    
    if exists == False:
        continue #Skip
    else: 
        # Do stuff
        parentData = session.query(Item).filter(Item.name.ilike(parentName)).first()
        parentId = parentData.id
        #print(parentId)
        
    exists = session.query(Item).filter(Item.name.ilike(compName)).scalar() is not None
    
    #Look up component item
    if exists == False:
        continue #Skip
    else:
        # Do component stuff
        compData = session.query(Item).filter(Item.name.ilike(compName)).first()
        compId = compData.id
    
    #Check if both exist in comp reltn already
    exists = session.query(compReltn).filter(compReltn.parent_item_id == parentId, compReltn.comp_item_id == compId).scalar() is not None
    
    if exists == False:
        #Do stuff here
        #INSERT
        newReltn = compReltn(parent_item_id = parentId, comp_item_id = compId, component_qty = compQty)
        session.add(newReltn)
        session.commit()
    else:
        #Get Qty
        existingData = session.query(compReltn).filter(compReltn.parent_item_id == parentId, compReltn.comp_item_id == compId).first()
        existingQty = existingData.component_qty
    #print(exists)
    print(f'parent: {parentId} {parentName} | component {compId} {compName} | Qty {compQty}')
    

parent: 33 SEEDS | component 14 WATER | Qty 0.1
parent: 10 APPLES | component 14 WATER | Qty 3.0
parent: 30 ORANGES | component 14 WATER | Qty 3.0
parent: 45 GRAPES | component 14 WATER | Qty 4.0
parent: 24 GRAIN | component 14 WATER | Qty 0.5
parent: 49 SUGARCANE | component 14 WATER | Qty 3.0
parent: 79 COTTON | component 14 WATER | Qty 1.0
parent: 50 WOOD | component 14 WATER | Qty 4.0
parent: 10 APPLES | component 33 SEEDS | Qty 1.0
parent: 30 ORANGES | component 33 SEEDS | Qty 1.0
parent: 45 GRAPES | component 33 SEEDS | Qty 1.0
parent: 24 GRAIN | component 33 SEEDS | Qty 1.0
parent: 49 SUGARCANE | component 33 SEEDS | Qty 1.0
parent: 79 COTTON | component 33 SEEDS | Qty 1.0
parent: 50 WOOD | component 33 SEEDS | Qty 1.0
parent: 33 SEEDS | component 19 TRANSPORT | Qty 0.1
parent: 10 APPLES | component 19 TRANSPORT | Qty 1.0
parent: 30 ORANGES | component 19 TRANSPORT | Qty 1.0
parent: 45 GRAPES | component 19 TRANSPORT | Qty 1.0
parent: 24 GRAIN | component 19 TRANSPORT | Qty 0.1


parent: 95 LAPTOPS | component 9 DISPLAYS | Qty 2.0
parent: 99 TELEVISIONS | component 18 PLASTIC | Qty 5.0
parent: 89 SMART-PHONES | component 28 ALUMINIUM | Qty 2.0
parent: 67 TABLETS | component 28 ALUMINIUM | Qty 3.0
parent: 95 LAPTOPS | component 18 PLASTIC | Qty 3.0
parent: 69 FABRIC | component 79 COTTON | Qty 2.0
parent: 22 UNDERWEAR | component 69 FABRIC | Qty 1.0
parent: 6 GLOVES | component 69 FABRIC | Qty 0.5
parent: 93 DRESS | component 69 FABRIC | Qty 3.0
parent: 86 SIMMI-SHOES | component 11 LEATHER | Qty 1.0
parent: 88 HANDBAGS | component 11 LEATHER | Qty 1.5
parent: 7 SNEAKERS | component 18 PLASTIC | Qty 1.0
parent: 90 GOLD-WATCH | component 51 GOLDEN-BARS | Qty 0.1
parent: 2 NECKLACE | component 51 GOLDEN-BARS | Qty 0.25
parent: 69 FABRIC | component 19 TRANSPORT | Qty 0.5
parent: 22 UNDERWEAR | component 19 TRANSPORT | Qty 1.0
parent: 6 GLOVES | component 19 TRANSPORT | Qty 1.0
parent: 93 DRESS | component 19 TRANSPORT | Qty 1.0
parent: 86 SIMMI-SHOES | component 1

In [86]:
test = session.query(compReltn).filter_by(parent_item_id=13).all()
for i in test:
    print(i.comp_item_id)

80
26
62
70
38


In [82]:
# clear table
session.query(compReltn).delete()

48

In [83]:
session.commit()